# Loading additional context item fields
The currently embedded sdk does not load all the properties of context items of a ContextHub view. This workaround shows how to extract more properties into a DataFrame by using API calls to retrieve the context items as a list of dicts, and then restructuring those dicts so they can be used to create a readable DataFrame.

A first step is to set the identifier of the ContextHub view we want to load. We can get this from the url of the ContextHub view. Since this notebook will load all context items into memory, rather than employ lazy loading, **running this notebook on ContextHub views with 10k+ items can cause performance issues and a crash of the notebook**.

In [1]:
view_id = "18093775-cc4c-4733-ba73-f583c1180c60"

We load the required packages, and use the token from our browser.

In [2]:
import os
import requests
import random
import pandas as pd
from datetime import datetime

token = os.environ["KERNEL_USER_TOKEN"]
headers = {'Authorization': f'Bearer {os.environ["KERNEL_USER_TOKEN"]}'}

We load the ContextHub view itself to retrieve its configuration:

In [3]:
r = requests.get(f'{os.environ["KERNEL_SERVER_URL"]}/context/view/{view_id}/enriched', headers=headers)

Then we can use the filter configruation to assemble the payload for the POST request to retrieve the items under that ContextHub view.

In [4]:
fetch_size = 1000

payload = {
    "fetchSize": fetch_size,
    "sortProperties": ["startEventDate"],
    "sortDirection": "asc",
    "filters": r.json()["data"]["filters"]
}

Since we can fetch only 1000 items at once, we need to iterate over the pages to make sure we fetch all items.

In [5]:
items = []
continuation_token = ""
while True:
    payload.update({"continuationToken": continuation_token})
    r = requests.post(f'{os.environ["KERNEL_SERVER_URL"]}/context/v2/item/search', headers=headers, json=payload)
    fetched_items = r.json()["content"]
    items += fetched_items
    if len(fetched_items) < fetch_size:
        break
    continuation_token = r.json()["page"]["continuationToken"]

We define restructuring function for context items. This function can be customized.

In [6]:
def restructure(item):
    '''Extract relevant properties from the context item json response to simple 1-level dict'''
    return {
        "start": datetime.strptime(item["events"][0]["occurred"], '%Y-%m-%dT%H:%M:%SZ'),
        "end": datetime.strptime(item["events"][-1]["occurred"], '%Y-%m-%dT%H:%M:%SZ'),
        "description": item["description"],
        "keywords": ", ".join(item["keywords"]),
        "user": item["userDetails"]["username"],
        "created": datetime.strptime(item["createdDate"], '%Y-%m-%dT%H:%M:%S.%fZ'),
        "key": item["shortKey"],
        "state": item["events"][-1].get("state"),
        **item["fields"],
    }

We can then simply create a DataFrame from the restructured items.

In [7]:
df = pd.DataFrame([restructure(item) for item in items])

In [8]:
df